In [ ]:
!pip install tensorflow-gpu

In [ ]:
import os
l1=len(os.listdir("./datasets/test_noisy"))
l2=len(os.listdir("./datasets/train_noisy"))

In [ ]:
import argparse
from model import *
from model_loss import *
from model_data import *
from model_module import *
from complex_layers.networks import *
from complex_layers.STFT import *
from complex_layers.activations import *

In [ ]:
def data_generator(train_arguments, test_arguments):
      train_generator = datagenerator(**train_arguments)

      test_generator  = datagenerator(**test_arguments)
      return train_generator, test_generator

@tf.function
def loop_train (model, optimizer, train_noisy_speech, train_clean_speech):
      tf.config.experimental_run_functions_eagerly(True)
      with tf.GradientTape() as tape:
            train_predict_speech = model(train_noisy_speech)
            if loss_function == "SDR":
                  train_loss = modified_SDR_loss(train_predict_speech, train_clean_speech)
            elif loss_function == "wSDR":
                  train_loss = weighted_SDR_loss(train_noisy_speech, train_predict_speech, train_clean_speech)

      gradients = tape.gradient(train_loss, model.trainable_variables)
      optimizer.apply_gradients(zip(gradients, model.trainable_variables))
      return train_loss

@tf.function
def loop_test (model, test_noisy_speech, test_clean_speech):
      'Test loop do not caclultae gradient and backpropagation'
      test_predict_speech = model(test_noisy_speech)
      if loss_function == "SDR":
            test_loss = modified_SDR_loss(test_predict_speech, test_clean_speech)
      elif loss_function == "wSDR":
            test_loss = weighted_SDR_loss(test_noisy_speech, test_predict_speech, test_clean_speech)

      return test_loss

def learning_rate_scheduler (epoch, learning_rate):
      if (epoch+1) <= int(0.5*epoch):
            return 1.00 * learning_rate
      elif (epoch+1) > int(0.5*epoch) and (epoch+1) <= int(0.75*epoch):
            return 0.20 * learning_rate
      else:
            return 0.05 * learning_rate
      
def model_flow (model, total_epochs, train_generator, test_generator,start=0):
      train_step = l2// batch_size
      test_step  = l1// batch_size
      start_epoch=start

      for epoch in tqdm(range (start_epoch,total_epochs)):
            train_batch_losses = 0
            test_batch_losses  = 0
            
            optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate_scheduler(epoch, learning_rate), beta_1 = 0.9)
            'Training Loop'
            for index, (train_noisy_speech, train_clean_speech) in tqdm(enumerate(train_generator),disable=True):
                  loss = loop_train (model, optimizer, train_noisy_speech, train_clean_speech)
                  train_batch_losses = train_batch_losses + loss

            'Test Loop'
            for index, (test_noisy_speech, test_clean_speech) in tqdm(enumerate(test_generator),disable=True):
                  loss  = loop_test (model, test_noisy_speech, test_clean_speech)
                  test_batch_losses  = test_batch_losses + loss

            'Calculate loss per batch data'
            train_loss = train_batch_losses / train_step
            test_loss  = test_batch_losses / test_step
           
            templet = "Epoch : {:3d},     TRAIN LOSS : {:.5f},     TEST LOSS  :  {:.5f}"
            print(templet.format(epoch+1, train_loss.numpy(), test_loss.numpy()))
            tmplet = "{:.5f} {:.5f}"
            line = tmplet.format(train_loss.numpy(), test_loss.numpy())
            outF = open("loss.txt", "a")
            outF.write(line)
            outF.write("\n")
            outF.close()
            if ((epoch+1) % 1) == 0: #save frequency was 10
                  model.save("./model_save/" + save_file_name + str(epoch+1) + ".h5")
            

In [ ]:
      tf.random.set_seed(seed = 42)
      parser = argparse.ArgumentParser(description = 'MODEL SETTING OPTION...')
      parser.add_argument("--model", type = str, default = "dcunet20", help = "model type")
      parser.add_argument("--epoch", type = int, default = 1000,        help = "Input epochs") #200
      parser.add_argument("--batch", type = int, default = 11,         help = "Input batch size") #64
      parser.add_argument("--loss",  type = str, default = "wSDR",     help = "Input Loss function") #wSDR
      parser.add_argument("--optim", type = str, default = "adam",     help = "Input optimizer option")
      parser.add_argument("--lr",    type = float, default = 0.0005,    help = "Inputs learning rate") #0.002
      parser.add_argument("--trn",   type = str, default = "./datasets/train_noisy/", help = "training noisy") #train_noisy5/
      parser.add_argument("--trc",   type = str, default = "./datasets/train_clean/", help = "training clean") #####change this
      parser.add_argument("--ten",   type = str, default = "./datasets/test_noisy/",  help = "testing noisy")
      parser.add_argument("--tec",   type = str, default = "./datasets/test_clean/",  help = "testing clean")
      parser.add_argument("--save",  type = str, default = "dcunet20_",                  help = "save model file name")
      args, unknown = parser.parse_known_args()
      model_type       = args.model
      total_epochs     = args.epoch
      batch_size       = args.batch
      loss_function    = args.loss
      optimizer_type   = args.optim
      learning_rate    = args.lr
      train_noisy_path = args.trn
      train_clean_path = args.trc
      test_noisy_path  = args.ten
      test_clean_path  = args.tec
      save_file_name   = args.save
      train_arguments = {"inputs_ids" : os.listdir(train_noisy_path), 
                        "outputs_ids" : os.listdir(train_clean_path),
                        "inputs_dir" : train_noisy_path, 
                        "outputs_dir" : train_clean_path,
                        "batch_size" : batch_size}
      test_arguments  = {"inputs_ids" : os.listdir(test_noisy_path), 
                        "outputs_ids" : os.listdir(test_clean_path),
                        "inputs_dir" : test_noisy_path,
                        "outputs_dir" : test_clean_path,
                        "batch_size" : batch_size}
      train_generator, test_generator = data_generator(train_arguments = train_arguments, test_arguments = test_arguments)
      if model_type == "naive_dcunet16":
            model = Naive_DCUnet16().model()
            #custom_objects = {"weighted_SDR_loss": weighted_SDR_loss,"complex_NaiveBatchNormalization": complex_NaiveBatchNormalization,"STFT_network": STFT_network,"complex_Conv2D": complex_Conv2D,"complex_BatchNorm2d":complex_BatchNorm2d,"complex_Conv2DTranspose":complex_Conv2DTranspose,"ISTFT_network":ISTFT_network}
            #with tf.keras.utils.custom_object_scope(custom_objects):
              #model=tf.keras.models.load_model('./model_save0/ndcunet16_500.h5')
      elif model_type == "naive_dcunet20":
            model = Naive_DCUnet20().model()
            #custom_objects = {"weighted_SDR_loss": weighted_SDR_loss,"complex_NaiveBatchNormalization": complex_NaiveBatchNormalization,"STFT_network": STFT_network,"complex_Conv2D": complex_Conv2D,"complex_BatchNorm2d":complex_BatchNorm2d,"complex_Conv2DTranspose":complex_Conv2DTranspose,"ISTFT_network":ISTFT_network}
            #with tf.keras.utils.custom_object_scope(custom_objects):
              #model=tf.keras.models.load_model('./model_save/ndcunet20_120.h5')
      elif model_type == "dcunet16":
            model = DCUnet16().model()
      elif model_type == "dcunet20":
            model = DCUnet20().model()
            #model=tf.keras.models.load_model('./model_save/dcunet20_16.h5')
            #custom_objects = {"weighted_SDR_loss": weighted_SDR_loss,"STFT_network": STFT_network,"complex_Conv2D": complex_Conv2D,"complex_BatchNorm2d":complex_BatchNorm2d,"complex_Conv2DTranspose":complex_Conv2DTranspose,"ISTFT_network":ISTFT_network}
            #with tf.keras.utils.custom_object_scope(custom_objects):
              #model=load_model_hdf5("model.h5")
              #tf.keras.models.load_model
              #model=tf.keras.models.load_model('./model_save/dcunet20_6.h5',compile=False)

In [ ]:
model.compile(optimizer='adam')

In [ ]:
pip install silence_tensorflow

  Created wheel for silence-tensorflow: filename=silence_tensorflow-1.1.1-cp37-none-any.whl size=3747 sha256=ded874d03b6b02cc062564de9938470967a3def235316413faa0ee0dc6c8db10
  Stored in directory: /root/.cache/pip/wheels/51/0b/35/cf3020764bee61daa81fa249df3a448e3806344a087fc12292
Successfully built silence-tensorflow


In [ ]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [ ]:
model_flow (model, total_epochs, train_generator, test_generator,7)


  0%|          | 0/993 [00:00<?, ?it/s]

In [ ]:
!nvidia-smi

Sun Jun  6 11:54:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------